In [ ]:
## Installation of Required Packages
!apt-get update
!apt-get install chromium-chromedriver -y
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
!pip install webdriver_manager

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import re, csv
import pandas as pd

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=options)

year = ['2017', '2018']
month = ['01','02','03','04','05','06','07','08','09','10','11','12']
summary = pd.DataFrame(columns=['date','rainfall'])

for y in year:
    for m in month:
        driver.get("https://www.hko.gov.hk/en/cis/dailyExtract.htm?y=" + y + "&m=" + str(int(m)))
        
        element = WebDriverWait(driver, 30).until(
            EC.visibility_of_element_located((By.XPATH,"//table[@id='t1']")),
        )
        
        html = driver.page_source
        bsObj = BeautifulSoup(html, 'html.parser')
        data_table = bsObj.findAll("table", id="t1")
        tb = pd.read_html(str(data_table))
        
        df = tb[0]
        rain = df.iloc[:,[0,8]]
        rain.columns = ['day', 'rainfall']
        rain = rain[rain.day.apply(lambda x: x.isnumeric())]
        rain.rainfall = rain.rainfall.replace({'Trace':'0.0'})
        rain['date'] = rain.day + "/" + m + "/" + y
        del rain['day']

        summary = summary.append(rain, ignore_index=True)

In [ ]:
summary.to_csv('rainfall.csv', index=False)